# Eniviroment setup

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.models import vit_b_16
from torchvision.transforms import transforms

from dataset.dataset_info import ClassifierDatasetInfo
from dataset.dataset_type import DatasetType
from dataset.STARCOP_dataset import STARCOPDataset

from models.Tools.FilesHandler.model_files_handler import ModelFilesHandler
from models.Tools.Measures.measure_tool_factory import MeasureToolFactory
from models.Tools.Measures.model_type import ModelType

## Setup datasets
STARCOPDataset is custom class that derives torch.utils.data.Dataset class. It's defined in dataset module.  

In [ ]:
def setup_dataloaders(data_path: str = r"data", batch_size: int = 32):
    train_dataset = STARCOPDataset(
        data_path=data_path,
        data_type=DatasetType.EASY_TRAIN,
        image_info_class=ClassifierDatasetInfo,
    )
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = STARCOPDataset(
        data_path=data_path,
        data_type=DatasetType.TEST,
        image_info_class=ClassifierDatasetInfo,
    )
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader

## Setup models

### Model class

In [ ]:
class CustomViT(nn.Module):
    def __init__(self, num_channels=9, num_classes=2):
        super(CustomViT, self).__init__()
        # Load pre-trained ViT model
        self.vit = vit_b_16(weights=None)  # Use pretrained weights if desired

        # Modify the input embedding layer to accept `num_channels`
        self.vit.conv_proj = nn.Conv2d(num_channels, self.vit.conv_proj.out_channels,
                                       kernel_size=self.vit.conv_proj.kernel_size,
                                       stride=self.vit.conv_proj.stride,
                                       padding=self.vit.conv_proj.padding,
                                       bias=(self.vit.conv_proj.bias is not None))

        # Modify the classifier head for binary classification
        self.vit.heads = nn.Sequential(
            nn.Linear(self.vit.heads.head.in_features, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.vit(x)

### Prepare models 

In [ ]:
def setup_model(lr: float, device: str):
    model = CustomViT().to(device)

    criterion = nn.CrossEntropyLoss()  # Binary classification
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return model, criterion, optimizer


## Prepare training function

In [ ]:
def train(criterion, device, epochs, model, optimizer, dataloader, transform):
    model.train()
    for epoch in range(epochs):  # Adjust the number of epochs
        running_loss = 0.0
        for batch_id, (images, mag1c, labels) in enumerate(dataloader):  # Assume a PyTorch DataLoader is set up
            optimizer.zero_grad()

            input_image = torch.cat((images, mag1c), dim=1)
            input_image = transform(input_image).to(device)
            labels = labels.long().to(device)

            outputs = model(input_image)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}")

## Prepare evaluate function

In [ ]:
def evaluate(criterion, device, model, dataloader, transform, measurer):
    model.eval()
    all_predictions = []
    all_labels = []
    running_loss = 0.0

    for batch_id, (images, mag1c, labels) in enumerate(dataloader):
        input_image = torch.cat((images, mag1c), dim=1)
        input_image = transform(input_image).to(device)
        labels = labels.long().to(device)

        outputs = model(input_image)
        predictions = torch.argmax(outputs, dim=1)
        loss = criterion(outputs, labels)

        running_loss += loss.item()
        all_predictions.append(predictions.cpu().detach())
        all_labels.append(labels.cpu().detach())

    measures = measurer.compute_measures(torch.cat(all_predictions), torch.cat(all_labels))
    print(f"Validation loss: {running_loss / len(dataloader)}.\nMeasures:\n{measures}")
    return measures

# Train model 

In [ ]:
epochs = 10
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-4

train_dataloader, test_dataloader = setup_dataloaders()
model, criterion, optimizer = setup_model(lr, device)
model_handler = ModelFilesHandler()
measurer = MeasureToolFactory.get_measure_tool(ModelType.TRANSFORMER)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.Normalize(mean=[0.5] * 9, std=[0.5] * 9)  # Normalize for 9 channels
])

train(criterion, device, epochs, model, optimizer, train_dataloader, transform)
measures = evaluate(criterion, device, model, test_dataloader, transform, measurer)


# Save model

In [ ]:
model_handler.save_model(
    model=model,
    metrics = measures,
    model_type=ModelType.TRANSFORMER_CLASSIFIER,
    epoch=epochs,
)